# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Generate a list of shipment numbers

In [131]:
def generate_shipment_numbers(shipment_number=340434188193323500, size=50): #340434188193324407
    import numpy as np
    shipment_numbers_list = []
    for i in np.arange(1, size):
        shipment_number = int(shipment_number) + i
        shipment_numbers = list(map(int, str(shipment_number)))
        shipment_number = str(shipment_number).rjust(20, '0')
        shipment_numbers_list.append(shipment_number)
    return shipment_numbers_list

In [132]:
shipment_numbers = generate_shipment_numbers(shipment_number=340434188193323500, size=50)
print("First elements of the list", shipment_numbers[0:5])

First elements of the list ['00340434188193323501', '00340434188193323503', '00340434188193323506', '00340434188193323510', '00340434188193323515']


#### Download the HTML content for a list of shipment numbers

In [133]:
def request(shipment_number="00340434188193323500", start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    import requests
    import numpy as np
    for shipment_number in shipment_numbers:
        request_url = start_url + shipment_number
    return requests.get(request_url).content

In [134]:
html_response = request(shipment_number="00340434188193323500")
print(html_response[0:50])

b'\n<div id="verfolgen-main-content">\n  \n\n\n\n\n\n\t\n\n\n<sc'


#### Parsing HTML and converting it into JSON

In [135]:
def html_to_json(html, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_numbers=None):
    from lxml import etree
    import json
    
    html_tree = etree.HTML(html)
    
    def clean_json(dirty_json):
        start = dirty_json.find('JSON.parse(')
        end = dirty_json.find('"),', start)
        cjson = dirty_json[start:end]
        cjson = cjson.replace('JSON.parse("', '')
        cjson = cjson.replace('\\', '')
        return cjson
    
    def find_json_element(html_tree):
        json_element = html_tree.xpath('//div')
        return str(etree.tostring(json_element[0]))
    
    dirty_json = find_json_element(html_tree)
    json_string = clean_json(dirty_json)
    return json.loads(json_string)

In [136]:
shipment_details_json = html_to_json(html_response)
print(shipment_details_json)

{'sendungen': [{'id': '00340434188193324725', 'hasCompleteDetails': True, 'sendungsinfo': {'gesuchteSendungsnummer': '00340434188193324725', 'sendungsrichtung': 'ANKOMMEND'}, 'sendungsdetails': {'sendungsnummern': {'sendungsnummer': '00340434188193324725'}, 'sendungsverlauf': {'fortschritt': 0, 'farbe': 0}}, 'sendungNichtGefunden': {'keineDatenVerfuegbar': True}}]}


#### Structure the shipment details in JSON format

In [137]:
def shipment_details(shipment_details_json, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number="00340434188193323500"):
    import datetime
    delivery_history_dict = {}
    delivery_history_dict["shipment_number"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
    delivery_history_dict["crawltime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    delivery_history_dict["url"] = start_url + shipment_number 
    delivery_history_dict["events"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])
    return delivery_history_dict

In [155]:
def drop_empty_values(shipment_details):
    shipment_history = shipment_details(shipment_details_json)
    for i in range(len(shipment_history)):
        if shipment_history[i]['events'] == None or []:
            del shipment_history[1]
        else:
            return shipment_history
    return 

In [156]:
import json
def save_dictionary(shipment_history=None, filename=None):
    print("Writing file {}".format(filename))
    with open(filename, 'w') as f:
        json.dump(shipment_history, f)

In [157]:
save_dictionary(shipment_history=None, filename="../data/00340434188193323500.json")

Writing file ../data/00340434188193323500.json


#### Main function, which goes through all shipment numbers and uses the previous methods

In [158]:
import time
def main():
    shipment_numbers = generate_shipment_numbers(shipment_number=340434188193323500, size=50)
    for shipment_number in shipment_numbers:
        time.sleep(0.5)
        print("Processing shipment number {}".format(shipment_number))
        html_response = request(shipment_number=shipment_number)
        shipment_details_json = html_to_json(html_response)
        shipment_history = shipment_details(shipment_details_json)
        save_dictionary(shipment_history, filename="../data/{}.json".format(shipment_number))
        print("\n")

In [159]:
main()

Processing shipment number 00340434188193323501
Writing file ../data/00340434188193323501.json


Processing shipment number 00340434188193323503
Writing file ../data/00340434188193323503.json


Processing shipment number 00340434188193323506
Writing file ../data/00340434188193323506.json


Processing shipment number 00340434188193323510
Writing file ../data/00340434188193323510.json


Processing shipment number 00340434188193323515
Writing file ../data/00340434188193323515.json


Processing shipment number 00340434188193323521
Writing file ../data/00340434188193323521.json


Processing shipment number 00340434188193323528
Writing file ../data/00340434188193323528.json


Processing shipment number 00340434188193323536
Writing file ../data/00340434188193323536.json


Processing shipment number 00340434188193323545
Writing file ../data/00340434188193323545.json


Processing shipment number 00340434188193323555
Writing file ../data/00340434188193323555.json


Processing shipment number 003